In [3]:
%config Completer.use_jedi = False
import ipywidgets as widgets
from IPython.display import display
import threading
import cv2
import numpy as np
from enum import Enum
from collections import deque
import imutils
import time
import json

class Mode(Enum):
    HSV_FINDER = 0
    BALL_TRACKER = 1
    QUIT = 2

In [4]:
isinstance(Mode.QUIT, Mode)

True

In [5]:
#implémanter threading un jour ?
class VideoPlayer:
    def __init__(self, source):
        self.cap = cv2.VideoCapture(source)
        self.paused = False
        self.frame = None
        self.loop = True
    
    def read(self):
        """if not self.cap.isopened():
            return False, None"""
        
        if self.paused:
            return True, self.frame

        ret, self.frame = self.cap.read()
        if self.loop and not ret:
            self.restart()
            ret, self.frame = self.cap.read()
        return ret, self.frame
        
    def pause(self):
        if self.frame is None:
            self.read()
        self.paused = not(self.paused)
        
    def stop(self):
        self.cap.release()
    
    def restart(self):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        #Si en pause, on change l'image renvoyée
        if self.paused:
            _, self.frame = self.cap.read()
    
    def get_time(self):
        return self.cap.get(cv2.CAP_PROP_POS_MSEC)
        
def convert_single_color(color, convertion_function):
    #Cette couleur est bien convertie, mais les entiers qu'elle contient sont du type np.uint8
    #On souhaite donc les convertir en le type int natif de python.
    converted_color = cv2.cvtColor(np.uint8([[color]]), convertion_function)[0][0]
    return tuple(int(elem) for elem in converted_color)

In [6]:
#Rajouter count ?
class Ball:
    maxlen = 16
    def __init__(self, name, lower_hsv = (0, 0, 0), higher_hsv = (179, 255, 255), data = dict()):
        self.name = name #Changer en "name" ?
        self.lower_hsv = tuple(lower_hsv)
        self.higher_hsv = tuple(higher_hsv)
        self.trail = deque(maxlen = self.maxlen)
        #Les fonctions de dessin d'opencv utilisent RGB
        hsv_trail_color = ((lower_hsv[0] + higher_hsv[0]) // 2, 255, 255)
        self.trail_color = convert_single_color(hsv_trail_color, cv2.COLOR_HSV2BGR)
        self.data = data
    
    def to_dict(self):
        return {"name" : self.name, "lower_hsv" : self.lower_hsv,
                "higher_hsv" : self.higher_hsv, "data" : self.data}

    @classmethod
    def from_dict(cls, d):
        return Ball(d["name"], lower_hsv=d["lower_hsv"], higher_hsv=d["higher_hsv"], data=d["data"])
    
    def __repr__(self):
        return "Ball(name = '{}', lower_hsv = {}, higher_hsv = {})".format(self.name, self.lower_hsv, self.higher_hsv)

In [7]:
debug_view = widgets.Output(layout={'border': '1px solid black'})

@debug_view.capture(clear_output=True)
def bad_callback(event):
    print('This is about to explode')
    return 1.0 / 0.0

button = widgets.Button(
    description='click me to raise an exception',
    layout={'width': '300px'}
)
button.on_click(bad_callback())
button

Button(description='click me to raise an exception', layout=Layout(width='300px'), style=ButtonStyle())

In [8]:
debug_view

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'error', 'ename': 'TypeError', 'evalu…

In [9]:
class BallTracker():
    def __init__(self, source=0, data_path=None, mode=Mode.HSV_FINDER):
        #self.dropdown.options est un tuple
        #Hypothèse : les clés de self.balls sont exactement les éléments de self.dropdown.options
        layout = widgets.Layout(width='50%')
        button_layout = widgets.Layout(width = '40px')
        
        #Widgets HSV_FINDER
        #Utiliser des IntRangeSlider ?
        h_low  = widgets.IntSlider(value=0, min=0, max=179, description='h_low:', layout=layout)
        s_low  = widgets.IntSlider(value=0, min=0, max=255, description='s_low:', layout=layout)
        v_low  = widgets.IntSlider(value=0, min=0, max=255, description='v_low:', layout=layout)
        h_high = widgets.IntSlider(value=179, min=0, max=179, description='h_high:', layout=layout)
        s_high = widgets.IntSlider(value=255, min=0, max=255, description='s_high:', layout=layout)
        v_high = widgets.IntSlider(value=255, min=0, max=255, description='v_high:', layout=layout)
        speed  = widgets.IntSlider(value=30, min=0, max=100, description='speed:', layout=layout)

        name_ball = widgets.Text(placeholder='Name ball color')
        add_ball = widgets.Button(icon='plus', layout=button_layout)
        remove_ball = widgets.Button(icon='remove', layout=button_layout)
        modify_ball = widgets.Button(icon='refresh', layout=button_layout)
        add_ball.on_click(lambda b : self.add_ball())
        remove_ball.on_click(lambda b : self.remove_ball())
        modify_ball.on_click(lambda b : self.modify_ball())
        
        dropdown = widgets.Dropdown(description='Balls :', options=[])
        dropdown.observe(lambda change : self.dropdown_observe(change), names="value")
        
        """upload_balls = widgets.Button(icon='upload', layout=button_layout)
        download_balls = widgets.Button(icon= 'download', layout=button_layout)
        
        upload_balls.on_click(lambda b : self.upload_balls())
        download_balls.on_click(lambda b : self.download_balls())"""
        
        #Widgets BALL_TRACKER
        #...
        
        #Widgets GENERAUX
        #...
        
        self.widgets = {"h_low" : h_low, "s_low" : s_low, "v_low" : v_low,
                       "h_high" : h_high, "s_high" : s_high, "v_high" : v_high,
                       "speed" : speed, "name_ball" : name_ball, "add_ball" : add_ball,
                       "remove_ball" : remove_ball, "modify_ball" : modify_ball,
                       "dropdown" : dropdown}
        self.dropdown = dropdown #A SUPPRIMER ? 
        self.balls = dict()
        
        self.only_video = False
        self.vp = None
        #self.vp = VideoPlayer(source=source)
        self.source = source
        self.mode = mode #Check mode ? isinstance(mode, Mode)
        self.switch_idx = 0 #A RESET
        self.switch_frames = [] #???
        self.window_name = 'frame'
        self.thread = None
        self.grab_coord = True
        
        if data_path is not None:
            self.load_balls(data_path)
        
    def get_ball_properties_from_widget(self):
        name = self.widgets["name_ball"].value
        lower_hsv  = (self.widgets["h_low"].value, self.widgets["s_low"].value,
                      self.widgets["v_low"].value)
        higher_hsv = (self.widgets["h_high"].value, self.widgets["s_high"].value,
                      self.widgets["v_high"].value)
        return name, lower_hsv, higher_hsv
    
    def add_ball(self):
        name, lower_hsv, higher_hsv = self.get_ball_properties_from_widget()
        if name in self.dropdown.options :
            print("{} existe déjà.".format(name))
            return
        self.balls[name] = Ball(name, lower_hsv=lower_hsv, higher_hsv=higher_hsv)
        self.dropdown.options = list(self.dropdown.options) + [name]
        self.dropdown.value = name
        print(self.balls)
    
    def remove_ball(self):
        #Principe : on supprime ou modifie le nom actuellement sélectionné.
        name, lower_hsv, higher_hsv = self.get_ball_properties_from_widget()
        if len(self.dropdown.options) == 0:
            return
        to_remove = self.dropdown.value
        self.balls.pop(name)
        self.dropdown.options = [elem for elem in self.dropdown.options if elem != to_remove]
        print(self.balls)
        
    def modify_ball(self):
        new_name, lower_hsv, higher_hsv = self.get_ball_properties_from_widget()
        old_name = self.dropdown.value
        if old_name != new_name and new_name in self.dropdown.options:
            print("{} existe déjà.".format(new_name))
            return
        ball = self.balls.pop(old_name)
        self.balls[new_name] = Ball(new_name, lower_hsv=lower_hsv, higher_hsv=higher_hsv)
        self.dropdown.options = [elem if elem != old_name else new_name for elem in self.dropdown.options]
        self.dropdown.value = new_name
        print(self.balls)
    
    def dropdown_observe(self, change):
        #ATTENTION, IL FAUT D'ABORD CHANGER LES VALEURS, PUIS A LA FIN CHANGER LE SLIDER !
        #SINON PB LORS DE L'UPDATE
        name = change['new']
        #A refaire plus proprement
        self.widgets["h_low"].value, self.widgets["s_low"].value, self.widgets["v_low"].value = self.balls[name].lower_hsv
        self.widgets["h_high"].value, self.widgets["s_high"].value, self.widgets["v_high"].value = self.balls[name].higher_hsv
        self.widgets["name_ball"].value = name
        
    def save_balls(self, path):
        data = [self.balls[name].to_dict() for name in self.dropdown.options]
        with open(path, 'w') as f:
            json.dump(data, f)
        
    def load_balls(self, path):
        with open(path, 'r') as f:
            data = json.load(f)
        options = []
        self.balls = dict()
        for ball_data in data:
            ball = Ball.from_dict(ball_data)
            self.balls[ball.name] = ball
            options.append(ball.name)
        self.dropdown.options = options
    
    def display(self):
        #Faire avec Out à l'avenir
        display(
            widgets.VBox([
                self.widgets["h_low"],
                self.widgets["s_low"],
                self.widgets["v_low"],
                self.widgets["h_high"],
                self.widgets["s_high"],
                self.widgets["v_high"],
                self.widgets["speed"], 
                widgets.HBox([
                    widgets.Label("Balle :"),
                    self.widgets["name_ball"],
                    self.widgets["add_ball"],
                    self.widgets["remove_ball"],
                    self.widgets["modify_ball"]
                ]),
                self.widgets["dropdown"]
            ])
        )
    
    def run(self):
        
        cv2.namedWindow(self.window_name, cv2.WINDOW_NORMAL)
        while True:
            #Choses à faire à chaque fois qu'on change de mode
            #...

            #Choix du mode
            if self.mode == Mode.HSV_FINDER:
                self.hsv_finder()
            elif self.mode == Mode.BALL_TRACKER:
                self.ball_tracker()
            elif self.mode == Mode.QUIT:
                break
            else:
                raise ValueError("{} n'est pas un mode reconnu.".format(self.mode))
        cv2.destroyWindow(self.window_name)
        self.vp.stop()
    
    #Rajouter choix du mode quand on peut load data.
    def start(self, threaded=True):
        self.vp = VideoPlayer(source=self.source)
        self.display()
        #self.mode = Mode.BALL_TRACKER
        if threaded:
            self.thread = threading.Thread(target=self.run)
            #thread = threading.Thread(target = self.test)
            self.thread.start()
            print("started")
        else:
            self.widgets['h_low'].value = 50
            print("started")
            self.run()
    
    def stop(self):
        self.mode = Mode.QUIT
    
    def hsv_finder(self): #Runs on thread
    
        def only_color(frame, color_ranges):
            h_low, s_low, v_low, h_high, s_high, v_high = color_ranges
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            lower = np.array([h_low, s_low, v_low])
            upper = np.array([h_high, s_high, v_high])
            mask = cv2.inRange(hsv, lower, upper)
            res = cv2.bitwise_and(frame,frame, mask= mask)
            return res, mask

        while self.mode == Mode.HSV_FINDER:
            ret, img = self.vp.read()
            if not ret:
                self.stop()
                break

            h_low  = self.widgets["h_low"].value
            s_low  = self.widgets["s_low"].value
            v_low  = self.widgets["v_low"].value
            h_high = self.widgets["h_high"].value
            s_high = self.widgets["s_high"].value
            v_high = self.widgets["v_high"].value
            speed  = self.widgets["speed"].value

            img2, mask = only_color(img, (h_low, s_low, v_low, h_high, s_high, v_high))
            img_to_draw = img2 if not(self.only_video) else img
            cv2.imshow(self.window_name, img_to_draw)

            k = cv2.waitKey(speed)
            if k == 27:
                self.stop()
                break #Plus besoin ?
            elif k == ord('p'):
                self.vp.pause()
            elif k == ord('t'):
                self.only_video = not(self.only_video)
            elif k == ord('m'):
                self.mode = Mode.BALL_TRACKER
    
    def ball_tracker(self): #Runs on thread

        while self.mode == Mode.BALL_TRACKER:
            ret, frame = self.vp.read()
            if not ret:
                self.stop()
                break
                
            video_time = self.vp.get_time()

            #frame = imutils.resize(frame, width=600)
            blurred = cv2.GaussianBlur(frame, (11, 11), 0)
            hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
            # construct a mask for the color "green", then perform a series of dilations
            # and erosions to remove any small blobs left in the mask
            for ball in self.balls.values():
                mask1 = cv2.inRange(hsv, ball.lower_hsv, ball.higher_hsv)
                mask2 = cv2.erode(mask1, None, iterations=2)
                mask3 = cv2.dilate(mask2, None, iterations=2)
                # find contours in the mask and initialize the current (x, y) center of the ball
                cnts = cv2.findContours(mask3.copy(), cv2.RETR_EXTERNAL,
                    cv2.CHAIN_APPROX_SIMPLE)
                cnts = imutils.grab_contours(cnts)
                center = None
                # only proceed if at least one contour was found
                if len(cnts) > 0:
                    # find the largest contour in the mask, then use it to compute the minimum 
                    # enclosing circle and centroid
                    c = max(cnts, key=cv2.contourArea)
                    ((x, y), radius) = cv2.minEnclosingCircle(c)
                    M = cv2.moments(c)
                    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                    # only proceed if the radius meets a minimum size
                    if radius > 10:
                        # draw the circle and centroid on the frame, then update the list of tracked points
                        if not self.only_video :
                            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                # update the points queue
                ball.trail.appendleft(center)
                
                if self.grab_coord:
                    ball.data[video_time] = center
                
                # loop over the set of tracked points
                for i in range(1, len(ball.trail)):
                    # if either of the tracked points are None, ignore them
                    if ball.trail[i - 1] is None or ball.trail[i] is None:
                        continue
                    # otherwise, compute the thickness of the line and draw the connecting lines
                    thickness = int(np.sqrt(Ball.maxlen / float(i + 1)) * 2.5)
                    if not self.only_video:
                        cv2.line(frame, ball.trail[i - 1], ball.trail[i], ball.trail_color, thickness)
            # show the frame to our screen
            t = [frame, blurred, hsv]
            cv2.imshow(self.window_name, t[self.switch_idx])
            k = cv2.waitKey(1) & 0xFF
            if k == 27:
                self.stop()
                break
            elif k == ord('p'):
                self.vp.pause()
            elif k == ord('s'):
                self.switch_idx = (self.switch_idx + 1) % len(t)
            elif k == ord('t'):
                self.only_video = not(self.only_video)
            elif k == ord('m'):
                self.mode = Mode.HSV_FINDER

In [10]:
test = BallTracker(source='chant_cut.mp4')
test.start(threaded=True)

started


In [21]:
test.load_balls('test.json')

In [16]:
for ball in test.balls.values():
    print(ball)

Ball(name = 'red', lower_hsv = [0, 111, 125], higher_hsv = [7, 184, 255])
Ball(name = 'yellow', lower_hsv = [17, 75, 145], higher_hsv = [38, 149, 207])
Ball(name = 'blue', lower_hsv = [95, 46, 55], higher_hsv = [119, 207, 194])


In [ ]:
b = widgets.Dropdown(options=[1, 2, 3])
b.observe(lambda change : print(change), names="value")
display(b)

In [130]:
b.options = [1, 2, 3, 4]

In [131]:
 b.value = 2

{'name': 'value', 'old': 1, 'new': 2, 'owner': Dropdown(index=1, options=(1, 2, 3, 4), value=2), 'type': 'change'}


In [ ]:
a = Test()
b = Test()
print({"a":a, "b":b})

In [88]:
debug_view

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'stream', 'text': "{'JV': {'lower_hsv…

In [17]:
class Test:
    def __init__(self):
        self.button = widgets.Button(layout={'width': '300px'})
        self.button.on_click(lambda b: self.oups())
        self.i = 0
    
    @debug_view.capture(clear_output=True)
    def oups(self):
        print('This is about to explode')
        tmp = 1.0 / 0.0
    
    def print_sleep(self):
        time.sleep(5.0)
        print('thread ', self.i)
    
    def start_thread(self):
        thread = threading.Thread(target = self.print_sleep)
        thread.start()
        print('started ', self.i)
    
    def display(self):
        display(self.button)
    
    def __repr__(self):
        return str(self.i)

a = Test()
a.start_thread()

started  0


In [18]:
a.i = 10

thread  10


In [82]:
debug_view

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'stream', 'text': 'This is about to e…

In [ ]:
self.only_video = False
self.vp = VideoPlayer(source=source)
self.mode = Mode.HSV_FINDER
self.switch_idx = 0 #A RESET
self.switch_frames = ???
self.window_name = 'frame'
self.data = dict() # A bien initialiser !

def hsv_finder(self): #Runs on thread
    
    def only_color(frame, color_ranges):
        h_low, s_low, v_low, h_high, s_high, v_high = color_ranges
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lower = np.array([h_low, s_low, v_low])
        upper = np.array([h_high, s_high, v_high])
        mask = cv2.inRange(hsv, lower, upper)
        res = cv2.bitwise_and(frame,frame, mask= mask)
        return res, mask
    
    while self.mode is Mode.HSV_FINDER:
        ret, img = self.vp.read()
        if not ret:
            self.mode = Mode.QUIT
            break
        
        h_low  = self.sliders["h_low"].value
        s_low  = self.sliders["s_low"].value
        v_low  = self.sliders["v_low"].value
        h_high = self.sliders["h_high"].value
        s_high = self.sliders["s_high"].value
        v_high = self.sliders["v_high"].value
        speed  = self.sliders["speed"].value

        img2, mask = only_color(img, (h_low, s_low, v_low, h_high, s_high, v_high))
        img_to_draw = img2 if not(self.only_video) else img
        cv2.imshow(self.window_name, img_to_draw)
        
        k = cv2.waitKey(speed)
        if k == 27:
            self.mode = Mode.QUIT
            break #Plus besoin ?
        elif k == ord('p'):
            self.vp.pause()
        elif k == ord('t'):
            self.only_video = not(self.only_video)
        elif k == ord('m'):
            self.mode = Mode.BALL_TRACKER
        
    """print('h_low, s_low, v_low, h_high, s_high, v_high\n{}, {}, {}, {}, {}, {}'.format(
        h_low, s_low, v_low, h_high, s_high, v_high))"""

    
def ball_tracker(self): #Runs on thread

    balls = [Ball(color="red", lower_hsv=(0, 111, 125), higher_hsv=(7, 184, 255)),
            Ball(color="yellow", lower_hsv=(17, 75, 145), higher_hsv=(38, 149, 207)),
            Ball(color="blue", lower_hsv=(95, 46, 55), higher_hsv=(119, 207, 194))]

    #ATTENTION A LOOP ! ET A NE PAS NOMMER UNE BALLE TIME ?
    self.data = {ball.color : {"x":[], "y":[]} for ball in balls}
    self.data["time"] = []

    while True:
        ret, frame = self.vp.read()
        if not ret:
            self.mode = Mode.QUIT
            break
        self.data["time"].append(self.vp.cap.get(cv2.CAP_PROP_POS_MSEC))

        frame = imutils.resize(frame, width=600)
        blurred = cv2.GaussianBlur(frame, (11, 11), 0)
        hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
        # construct a mask for the color "green", then perform a series of dilations
        # and erosions to remove any small blobs left in the mask

        for ball_name, ball in self.balls.items():
            mask1 = cv2.inRange(hsv, ball.lower_hsv, ball.higher_hsv)
            mask2 = cv2.erode(mask1, None, iterations=2)
            mask3 = cv2.dilate(mask2, None, iterations=2)
            # find contours in the mask and initialize the current (x, y) center of the ball
            cnts = cv2.findContours(mask3.copy(), cv2.RETR_EXTERNAL,
                cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
            center = None
            # only proceed if at least one contour was found
            if len(cnts) > 0:
                # find the largest contour in the mask, then use it to compute the minimum 
                # enclosing circle and centroid
                c = max(cnts, key=cv2.contourArea)
                ((x, y), radius) = cv2.minEnclosingCircle(c)
                self.data[ball.color]["x"].append(x)
                self.data[ball.color]["y"].append(y)
                self.data[ball.color]["time"].append(self.vp.cap.get(cv2.CAP_PROP_POS_MSEC))
                M = cv2.moments(c)
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                # only proceed if the radius meets a minimum size
                if radius > 10:
                    # draw the circle and centroid on the frame, then update the list of tracked points
                    if not self.only_video :
                        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            # update the points queue
            ball.trail.appendleft(center)
            # loop over the set of tracked points
            for i in range(1, len(ball.trail)):
                # if either of the tracked points are None, ignore them
                if ball.trail[i - 1] is None or ball.trail[i] is None:
                    continue
                # otherwise, compute the thickness of the line and draw the connecting lines
                thickness = int(np.sqrt(Ball.maxlen / float(i + 1)) * 2.5)
                if not self.only_video:
                    cv2.line(frame, ball.trail[i - 1], ball.trail[i], ball.trail_color, thickness)
        # show the frame to our screen
        t = [frame, blurred, hsv, mask1, mask2, mask3]
        cv2.imshow("frame", t[switch])
        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break
        if key == ord('p'):
            self.vp.pause()
        if key == ord('s'):
            self.switch = (self.switch + 1) % len(t)
        if key == ord('t'):
            self.only_video = not(self.only_video)

def run(self):
    cv2.namedWindow(self.window_name, cv2.WINDOW_NORMAL)
    while True:
        #Choses à faire à chaque fois qu'on change de mode
        #...
        
        #Choix du mode
        if self.mode is Mode.HSV_FINDER:
            self.hsv_finder()
        elif self.mode is Mode.BALL_TRACKER:
            self.ball_tracker()
        elif self.mode is Mode.QUIT:
            break
        else:
            raise ValueError("{} n'est pas un mode reconnu.".format(self.mode))
    cv2.destroyWindow(self.window_name)
    self.vp.stop()
    
def start(self):
    thread = threading.Thread(target=self.run)
    thread.start()

In [ ]:
#A rendre fonctionnel un jour avec des vidéos (en plus de webcam)